# Лабораторная работа №5

## Imports

In [30]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BertTokenizer, BertModel
from scipy.spatial.distance import cosine

import pandas as pd
import numpy as np

In [19]:
DEVICE = torch.device('cuda:0')
PATH_READER = "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
PATH_RETRIEVAL = "DeepPavlov/rubert-base-cased"

## Dataset

In [5]:
""" 
    Загрузка даатасета
"""
dataset = load_dataset("sberquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

In [6]:
""" 
    Оформляем датасет в DataFrame
"""
df = pd.DataFrame(dataset['train'])
df

,id,title,context,question,answers
0,62310,SberChallenge,В протерозойских отложениях органические остат...,чем представлены органические остатки?,{'text': ['известковыми выделениями сине-зелён...
1,28101,SberChallenge,В протерозойских отложениях органические остат...,что найдено в кремнистых сланцах железорудной ...,"{'text': ['нитевидные водоросли, грибные нити'..."
2,48834,SberChallenge,В протерозойских отложениях органические остат...,что встречается в протерозойских отложениях?,"{'text': ['органические остатки'], 'answer_sta..."
3,83056,SberChallenge,В протерозойских отложениях органические остат...,что относится к числу древнейших растительных ...,{'text': ['скопления графито-углистого веществ...
4,5816,SberChallenge,В протерозойских отложениях органические остат...,как образовалось графито-углистое вещество?,{'text': ['в результате разложения Corycium en...
...,...,...,...,...,...
45323,6601,SberChallenge,"Познакомившись с двигателем Ленуара, осенью 18...",Когда подали заявку на патент на двигатель с ж...,"{'text': ['в январе 1861 года'], 'answer_start..."
45324,84192,SberChallenge,"Познакомившись с двигателем Ленуара, осенью 18...",Что создал после отклонения заявки Николаус Ау...,{'text': ['двухтактный атмосферный двигатель в...
45325,38284,SberChallenge,Главную роль в истории с письменным обязательс...,Что было целью разыгранного Веберами спектакля?,"{'text': ['сближение Моцарта с Констанцией'], ..."
45326,73427,SberChallenge,Главную роль в истории с письменным обязательс...,Что не мог подписать Моцарт из-за сильно разви...,"{'text': ['заявление'], 'answer_start': [376]}"


## Reader

In [4]:
""" 
    Загрузка reader модели
"""
reader_tokenizer = AutoTokenizer.from_pretrained(PATH_READER)
reader_model = AutoModelForQuestionAnswering.from_pretrained(PATH_READER)

tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [7]:
""" 
Функция reader

    Принимает:
        question :(str) - вопрос
        paragraph :(str) - контекст

    Возвращает:
        correct_answer :(str) - ответ
"""
def reader(question, paragraph):
    encoding = reader_tokenizer.encode_plus(text = question, text_pair = paragraph)
    inputs = encoding['input_ids']
    tokens = reader_tokenizer.convert_ids_to_tokens(inputs)
    
    output = reader_model(input_ids=torch.tensor([inputs]))
    start_index = torch.argmax(output[0])
    end_index = torch.argmax(output[1])

    answer = ' '.join(tokens[start_index:end_index + 1])
    correct_answer = answer.replace(' ','')
    correct_answer = correct_answer.replace('▁',' ')
    return correct_answer


### Test reader

In [8]:
df['context'][0]

'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.'

In [9]:
df['question'][0]


'чем представлены органические остатки?'

In [10]:
answer = reader(df['question'][0], df['context'][0])
print(answer)

 известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных


## Retrieval

In [35]:
""" 
    Загрузка модели - retrieval
"""
retrieval_tokenizer = BertTokenizer.from_pretrained(PATH_RETRIEVAL, do_lower_case = True)
retrieval_model = BertModel.from_pretrained(PATH_RETRIEVAL, output_hidden_states = True)
retrieval_model.to(DEVICE)
retrieval_model.eval()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [59]:
""" 
    Набор уникальных данных
"""
subsample = df.drop_duplicates('context')
paragraphs = subsample['context'].values

In [36]:
""" 
Функция vectors_from_texts

  Принимает:
    text :(str) - строка

  Возвращает:
    sentence_embedding :(np.array) - вектор на выходе Bert
"""
def vectors_from_texts(text):
  marked_text = "[CLS] " + text +" [SEP]"
  tokenized_text = retrieval_tokenizer.tokenize(marked_text)
  if len(tokenized_text) > 512:
    marked_text = "[CLS] " + text +" [SEP]"
    tokenized_text = retrieval_tokenizer.tokenize(marked_text)

  indexed_tokens = retrieval_tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)

  tokens_tensors = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  tokens_tensors = tokens_tensors.to(DEVICE)
  segments_tensors = segments_tensors.to(DEVICE)

  with torch.no_grad():
    outputs = retrieval_model(tokens_tensors, segments_tensors)
    hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim = 0)
  token_embeddings = torch.squeeze(token_embeddings, dim = 1)
  token_embeddings = token_embeddings.permute(1, 0, 2)

  token_vecs_sum = []
  for token in token_embeddings:
    sum_vec = torch.sum(token[-4:], dim = 0)
    token_vecs_sum.append(sum_vec)
    token_vecs = hidden_states[-2][0]
    
  sentence_embedding = torch.mean(token_vecs, dim = 0)
  return sentence_embedding.cpu().numpy()

In [69]:
""" 
Функция retrieval

  Принимает:
      question :(str) - вопрос

  Возвращает:
      id :(int) - индекс контекста
"""
def retrieval(question):
  question = vectors_from_texts(question)

  context_vectors = []
  for par in paragraphs:
    context_vectors.append(vectors_from_texts(par[:512]))
    
  results = []
  for vec in context_vectors:
    results.append(cosine(vec, question))
  return np.argmin(results)

### Retrieval test

In [83]:
question = df['question'].sample(1)
question = question.values[0]
print(question)

В США изобретателем радио считают ?


In [84]:
text = retrieval(question)

In [85]:
print(paragraphs[text])

Со вспышкой гражданской войны оперативная связь с дальним западом стала более необходима. Единственным возможным вариантом связи с землями за пределами реки Миссури был пони-экспресс. Используя такой способ передачи сообщений, требовалось 10 дней для доставки телеграмм и писем от Сент Джозефа до Сакраменто, Калифорния. В 1860 году Конгресс выпускает Pacific Telegraph Pact, предписывающий организацию тендера на создание трансконтинентальной линии. Несмотря на то, что телеграфная линия была чрезвычайно необходима, сама затея о её строительстве казалась невозможной, так как предстояло протянуть 2000 миль по горам и равнинам. Другие телеграфные компании отказались принимать участие в этом мероприятии. Даже президент Авраам Линкольн тогда говорил президенту компании Хайраму Сибли о том, что это безумная затея, так как даже если линию удастся завершить, то индейцы её разрушат. Инженеры предрекали проекту срок в 10 лет, однако Эдвард Крэйтон — молодой и находчивый агент компании Вестерн Юнион

## MiniChatGPT

In [74]:
""" 
    Тестовая выборка из вопросов и контекста для этих вопросов
"""
questions = [
    'Когда были установлены пенсии за спортивные достижения и выслугу лет ',
    'Что подразумевают под швейцарской культурой ?',
    'На какой период был арендован Россией комплекс Байконур '
    ]
contexts = [
    'Пенсии за спортивные достижения и выслугу лет были установлены Постановлением Совета Министров СССР,ВЦСПС и ЦК ВЛКСМ от 2 августа 1988 года № 945 О совершенствовании управления футболом, другими игровыми видами спорта и дополнительных мерах по упорядочению содержания команд и спортсменов по основным видам спорта . Согласно принятому в феврале 1989 года Порядку назначения и выплаты пенсий за выслугу лет заслуженным мастерам спорта СССР, мастерам спорта СССР международного класса — членам сборных команд СССР эти пенсии назначались при общем стаже работы не менее 20 лет заслуженным мастерам спорта СССР и мастерам спорта СССР международного класса, состоявшим в штате команд мастеров не менее 10 лет, в том числе в сборных командах СССР, или не менее 6 лет в составе сборных команд СССР .',
    'Культура страны развивалась, с одной стороны, под влиянием немецкой, французской и итальянской культуры и, с другой стороны, на основе особенного самосознания каждого кантона. Поэтому до сих пор очень трудно точно сказать, что такое собственно швейцарская культура . В самой Швейцарии различают швейцарскую культуру (как правило — фольклор) и культуру из Швейцарии — все имеющиеся жанры, в которых работают люди со швейцарским паспортом. Так, например, объединения музыкантов, играющих на альпенгорнах — это скорее швейцарская культура , а рок-группы Yello , Gotthard , Krokus и Samael — это культура из Швейцарии.',
    'Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.']

In [75]:
""" 
Функция MyChatGPT

    Принимает:
        question :(str) - вопрос

    Возвращает:
        answer :(str) - ответ
        paragraphs[id] :(str) - контекст, в котором находили ответ на вопрос
"""
def MyChatGPT(question):
  id = retrieval(question)
  answer = reader(question, paragraphs[id])
  return answer, paragraphs[id]

### Test ChatGpt

In [77]:
""" 
  Выводит результат работы MyChatGPT в виде ответ - контекст
"""
for i in range(len(questions)):
  print(MyChatGPT(questions[i]))

(' пенсии', 'Система ТАРГЕТ повышает ликвидность, являющуюся необходимым условием системы расчётов. Она предусматривает возможность получения её участниками дополнительных ликвидных средств, которые могут быть использованы для осуществления платежей. Центральные банки предоставляют беспроцентные дневные кредиты под соответствующее обеспечение, которые могут быть использованы в течение рабочего дня неоднократно. В качестве обеспечения принимаются все активы, которые используются при проведении операций рефинансирования.')
(' французской и немецкой частями Швейцарии', 'Взаимоотношения между французской и немецкой частями Швейцарии являются важнейшим фактором в развитии национальной истории. Однако они далеки от идеала. Отношения между основными культурно-языковыми ареалами страны с начала XIX века, когда к территории Швейцарии были присоединены густонаселенные франкоязычные области, и по сей день характеризуются наличием большого числа конфликтов и противоречий. Существует даже воображае